네, 모든 요구사항을 통합하고 예상되는 예외 상황 처리까지 포함하여 고도화된 최종 파이썬 코드를 작성해 드리겠습니다.

이 코드는 데이터의 구조를 외부 `schema.json` 파일에서 정의하고, 이를 동적으로 코드에 주입하여 사용합니다. 또한 배치/단일 저장, 덮어쓰기/이어쓰기 모드를 모두 지원하며, 발생할 수 있는 오류에 대한 방어 로직을 포함하고 있습니다.

-----

### 최종 완성 코드

#### 1\. 데이터 구조 정의 파일 (`schema.json`)

먼저, 코드와 같은 위치에 아래 내용으로 `schema.json` 파일을 생성해야 합니다. 이 파일이 데이터의 '설계도' 역할을 합니다.

```json
{
  "cn": "str",
  "title": "str",
  "abstract": "str",
  "embedding": "List[float]"
}
```

-----

#### 2\. 파이썬 전체 코드 (`main.py`)

아래 코드는 위 `schema.json` 파일을 읽어 동적으로 클래스를 생성하고, 모든 저장 로직을 수행합니다.

```python
import csv
import json
import os
from dataclasses import make_dataclass, fields, is_dataclass
from typing import List, Any, Dict

# --- 1. 동적 클래스 생성을 위한 헬퍼 ---

# JSON의 타입 문자열을 실제 파이썬 타입으로 변환하는 맵
TYPE_MAPPING = {
    'str': str,
    'int': int,
    'float': float,
    'bool': bool,
    'List[str]': List[str],
    'List[int]': List[int],
    'List[float]': List[float],
    'Dict': Dict,
    'Any': Any
}

def create_class_from_schema(class_name: str, schema_path: str = "schema.json") -> type:
    """
    JSON 스키마 파일로부터 동적으로 데이터 클래스를 생성합니다.

    Args:
        class_name (str): 생성할 클래스의 이름 (예: "Document").
        schema_path (str): 스키마 파일의 경로.

    Returns:
        type: 동적으로 생성된 데이터 클래스.
        
    Raises:
        FileNotFoundError: 스키마 파일이 존재하지 않을 경우.
        TypeError: 스키마에 지원하지 않는 타입이 정의된 경우.
    """
    try:
        with open(schema_path, 'r', encoding='utf-8') as f:
            schema = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"오류: 스키마 파일 '{schema_path}'을 찾을 수 없습니다.")
    except json.JSONDecodeError:
        raise ValueError(f"오류: '{schema_path}' 파일이 올바른 JSON 형식이 아닙니다.")

    fields_for_class = []
    for field_name, field_type_str in schema.items():
        actual_type = TYPE_MAPPING.get(field_type_str)
        if actual_type is None:
            raise TypeError(f"스키마 오류: 지원하지 않는 타입 '{field_type_str}'입니다.")
        fields_for_class.append((field_name, actual_type))
    
    # make_dataclass를 사용하여 동적으로 데이터 클래스 생성
    return make_dataclass(class_name, fields_for_class)


# --- 2. 핵심 CSV 저장 로직 ---

def save_documents_batch(
    documents: List[Any],
    output_file: str,
    document_class: type,
    mode: str = 'w'
):
    """
    여러 데이터 객체를 CSV 파일에 배치 저장합니다.

    Args:
        documents (List[Any]): 저장할 객체들의 리스트.
        output_file (str): 저장할 파일 경로.
        document_class (type): 데이터 객체의 타입 (동적으로 생성된 클래스).
        mode (str): 파일 저장 모드. 'w' (덮어쓰기) 또는 'a' (이어쓰기).
    """
    # --- 입력값 유효성 검사 ---
    if not documents:
        print("경고: 저장할 데이터가 없어 작업을 중단합니다.")
        return
    if not is_dataclass(document_class):
        raise TypeError("오류: 'document_class'는 데이터 클래스 타입이어야 합니다.")
    if mode not in ['w', 'a']:
        raise ValueError("오류: mode는 'w' 또는 'a'여야 합니다.")
    
    for doc in documents:
        if not isinstance(doc, document_class):
            raise TypeError(f"오류: 저장할 데이터는 모두 '{document_class.__name__}' 타입이어야 합니다.")

    # --- 파일 처리 ---
    file_exists = os.path.exists(output_file)
    headers = [field.name for field in fields(document_class)]

    try:
        with open(output_file, mode, newline="", encoding="utf-8") as f:
            writer = csv.writer(f)

            # 'w' 모드이거나, 'a' 모드인데 파일이 새로 생성될 때만 헤더 작성
            if mode == 'w' or not file_exists:
                writer.writerow(headers)

            for doc in documents:
                row_data = [getattr(doc, header) for header in headers]
                writer.writerow(row_data)

    except IOError as e:
        raise IOError(f"파일 쓰기 오류: '{output_file}' 파일에 쓸 수 없습니다. 권한을 확인하세요. ({e})")
    
    print(f"✅ 총 {len(documents)}개 항목을 '{output_file}'에 '{mode}' 모드로 저장했습니다.")


def save_document_single(
    document: Any,
    output_file: str,
    document_class: type,
    mode: str = 'w'
):
    """
    하나의 데이터 객체를 CSV 파일에 저장합니다. (내부적으로 batch 함수 재사용)
    """
    # 단일 저장은 "객체가 1개인 배치 저장"과 동일하므로, batch 함수를 호출
    save_documents_batch([document], output_file, document_class, mode=mode)


# --- 3. 실제 사용 예시 ---

if __name__ == "__main__":
    try:
        # 1. 외부 스키마로부터 'Document' 클래스를 동적으로 생성
        DynamicDocument = create_class_from_schema("Document", "schema.json")
        print(f"'{'schema.json'}'으로부터 '{DynamicDocument.__name__}' 클래스를 성공적으로 생성했습니다.")
        print("-" * 30)

        # 2. 동적으로 생성된 클래스를 사용하여 데이터 객체 생성
        docs_1 = [
            DynamicDocument("doc1", "첫 번째 문서", "요약1", [0.1, 0.2]),
            DynamicDocument("doc2", "두 번째 문서", "요약2", [0.3, 0.4]),
        ]
        doc_3 = DynamicDocument("doc3", "세 번째 문서", "요약3", [0.5, 0.6])
        docs_4 = [
            DynamicDocument("doc4", "네 번째 문서", "요약4", [0.7, 0.8]),
            DynamicDocument("doc5", "다섯 번째 문서", "요약5", [0.9, 1.0]),
        ]
        
        output_filename = "documents_output.csv"

        # 3. 시나리오별 함수 호출
        print("\n--- 시나리오 1: 배치 데이터로 새 파일 생성 (mode='w') ---")
        save_documents_batch(docs_1, output_filename, DynamicDocument, mode='w')

        print("\n--- 시나리오 2: 기존 파일에 단일 데이터 이어쓰기 (mode='a') ---")
        save_document_single(doc_3, output_filename, DynamicDocument, mode='a')

        print("\n--- 시나리오 3: 기존 파일에 배치 데이터 이어쓰기 (mode='a') ---")
        save_documents_batch(docs_4, output_filename, DynamicDocument, mode='a')
        
        print("\n--- 최종 파일 내용 확인 ---")
        with open(output_filename, 'r', encoding='utf-8') as f:
            print(f.read().strip())
        print("-" * 30)
        
        print("\n--- 시나리오 4: 잘못된 타입의 데이터를 저장 시도 (오류 발생) ---")
        class WrongType: pass
        wrong_doc = WrongType()
        save_document_single(wrong_doc, "temp.csv", DynamicDocument)

    except (FileNotFoundError, TypeError, ValueError, IOError) as e:
        print(f"\n💥 프로그램 실행 중 오류가 발생했습니다: {e}")

```

### 코드의 주요 특징 및 고도화된 부분

  * **완벽한 관심사 분리**:

      * **데이터 구조 (`schema.json`)**: 데이터의 형태는 JSON 파일에서 독립적으로 관리됩니다.
      * **클래스 생성 (`create_class_from_schema`)**: 스키마를 실제 파이썬 클래스로 변환하는 역할만 담당합니다.
      * **저장 로직 (`save_...`)**: 생성된 클래스의 내부 구조를 몰라도, 주입된 타입에 맞춰 유연하게 동작합니다.

  * **강력한 오류 처리**:

      * `schema.json` 파일이 없거나, JSON 형식이 잘못되었거나, 지원하지 않는 타입을 사용했을 경우를 대비해 명확한 오류 메시지를 출력합니다.
      * `save_documents_batch` 함수는 저장하려는 데이터가 주입된 클래스 타입과 일치하는지 `isinstance`로 검사하여 데이터 무결성을 보장합니다.
      * 파일 쓰기 권한이 없는 등의 `IOError` 발생 시에도 프로그램을 안전하게 중단하고 원인을 알려줍니다.

  * **코드 재사용성 (DRY 원칙)**:

      * `save_document_single` 함수가 별도의 로직 없이 `save_documents_batch` 함수를 재사용하여 코드 중복을 피하고 유지보수성을 높였습니다.

  * **명확한 실행 흐름**:

      * `if __name__ == "__main__":` 블록을 사용하여, 이 파일이 직접 실행될 때만 예시 코드가 동작하도록 구성했습니다.
      * 시나리오별로 어떤 작업을 수행하는지 `print` 문으로 명확하게 안내하여 코드의 동작을 쉽게 이해할 수 있습니다.

이 코드는 단순히 기능 구현을 넘어, 안정성과 유연성, 확장성을 모두 고려한 잘 설계된 구조입니다.